### K-Fold Cross Validation : K겹 교차검증
- 데이터를 K개로 분할
- k-1개를 학습용 데이터로, 1개를 평가용 데이터로 사용
- 이를 K번 반복하여 K개의 성능 지표를 얻어내는 방법
<img src="https://imghub.insilicogen.com/media/photos/cv.png">
- sklearn.model_selection.cross_val_score(estimator #학습/평가할모델, X #입출력을 포함하는 데이터, y=None #타겟데이터 cv=None #K를 의미)

### 언제사용하는가?
### A. 과적합 방지

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score #Confusion matrix 수업 때 진행할 예정 
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score 
import numpy as np
import pandas as pd

C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
### 과적합의 발생 경로 살펴보기
fold_iris = load_iris()
features = fold_iris.data #입출력 데이터
label= fold_iris.target #타겟데이터

In [5]:
#훈련 검사 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=111)

#결정트리 모델을 선택하여 학습과 예측을 진행
model = DecisionTreeClassifier()
model.fit(X_train, y_train) #모델학습
pred_train = model.predict(X_train) #train 예측
pred_test = model.predict(X_test) #test예측
print(accuracy_score(y_train, pred_train)) #훈련데이터 정확도 
print(accuracy_score(y_test, pred_test)) #테스트 데이터 정확도

1.0
0.9


In [9]:
#깊이를 제한한 결정트리 모델을 이용
model_md = DecisionTreeClassifier(random_state=111, max_depth=3)
model_md.fit(X_train, y_train)
pred_train_md = model_md.predict(X_train)
pred_test_md = model_md.predict(X_test)
print(accuracy_score(y_train,pred_train_md))
print(accuracy_score(y_test, pred_test_md))

0.9916666666666667
0.9


- 둘 모두 훈련데이터의 정확도가 테스트 데이터의 정확도보다 높은 과적합이 발생


### 교차검증으로 과적합 해소하기

In [11]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5) #데이터셋 분리
cv_accuracy_train = []
cv_accuracy_test =[]
kf_model = DecisionTreeClassifier(random_state=111,max_depth=3)

In [34]:
n_iter = 0 #5번 진행하니 초기값 설정
for train_idx, test_idx in kfold.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습을 위해서 dt 배우기
    kf_model.fit(X_train, y_train)
    #예측
    kf_pred_train = kf_model.predict(X_train)
    kf_pred_test = kf_model.predict(X_test)
    #정확도를 5번 측정하면서 대입
    n_iter += 1
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train), 4)
    accuracy_test = np.round(accuracy_score(y_test, kf_pred_test), 4)
    
    print("\n{} 번 train 교차 검증 정확도: {}, test 교차 검증 정확도 :{} ".format(n_iter, accuracy_train, accuracy_test))
    print("학습레이블 분포 : ", collections.Counter(y_train), "\n검사레이블 분포 : ", collections.Counter(y_test))
    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
    
print('train 평균 정확도', np.mean(cv_accuracy_train))
print('test 평균 정확도', np.mean(cv_accuracy_test))



1 번 train 교차 검증 정확도: 0.975, test 교차 검증 정확도 :1.0 
학습레이블 분포 :  Counter({1: 50, 2: 50, 0: 20}) 
검사레이블 분포 :  Counter({0: 30})

2 번 train 교차 검증 정확도: 0.975, test 교차 검증 정확도 :0.9667 
학습레이블 분포 :  Counter({2: 50, 1: 40, 0: 30}) 
검사레이블 분포 :  Counter({0: 20, 1: 10})

3 번 train 교차 검증 정확도: 0.9917, test 교차 검증 정확도 :0.8333 
학습레이블 분포 :  Counter({0: 50, 2: 50, 1: 20}) 
검사레이블 분포 :  Counter({1: 30})

4 번 train 교차 검증 정확도: 0.9833, test 교차 검증 정확도 :0.9333 
학습레이블 분포 :  Counter({0: 50, 1: 40, 2: 30}) 
검사레이블 분포 :  Counter({2: 20, 1: 10})

5 번 train 교차 검증 정확도: 0.975, test 교차 검증 정확도 :0.7 
학습레이블 분포 :  Counter({0: 50, 1: 50, 2: 20}) 
검사레이블 분포 :  Counter({2: 30})
train 평균 정확도 0.9800000000000002
test 평균 정확도 0.8866599999999999


Kfold의 한계 : 랜덤 데이터 추출을해서 정답 비중에 고르게 분포하지 않을 수도 있음

### 교차검증 정확도가 1도 나옴
=> 검사 레이블이 하나만 나올때도 있어서 그럼
- 이를 해소하기위해 StratifieldKFold를 진행한다

In [35]:
from sklearn.model_selection import StratifiedKFold

skf_iris = StratifiedKFold(n_splits=3)
cnt_iter=0
n_iter=0

skf_cv_accuracy_train = []
skf_cv_accuracy_test = []
skf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

for train_idx, test_idx in skf_iris.split(features, label):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습을 위해서 dt배우기
    skf_model.fit(X_train, y_train)
    #예측
    skf_pred_train = skf_model.predict(X_train)
    skf_pred_test = skf_model.predict(X_test)
    
    #정확도를 5번 측정하면서 값에 넣을 것이니
    n_iter +=1
    accuracy_train = np.round(accuracy_score(y_train, skf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, skf_pred_test),4)
    
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))
    print("학습레이블 분포 : ", collections.Counter(y_train), "\n검사레이블 분포 : ", collections.Counter(y_test))

    skf_cv_accuracy_train.append(accuracy_train)
    skf_cv_accuracy_test.append(accuracy_test)
    
print('train 평균 정확도',np.mean(skf_cv_accuracy_train))
print('test 평균 정확도',np.mean(skf_cv_accuracy_test))


1 번 train 교차 검증 정확도 :0.95 ,test 교차 검증 정확도 :0.98 
학습레이블 분포 :  Counter({2: 34, 0: 33, 1: 33}) 
검사레이블 분포 :  Counter({0: 17, 1: 17, 2: 16})

2 번 train 교차 검증 정확도 :0.99 ,test 교차 검증 정확도 :0.94 
학습레이블 분포 :  Counter({1: 34, 0: 33, 2: 33}) 
검사레이블 분포 :  Counter({0: 17, 2: 17, 1: 16})

3 번 train 교차 검증 정확도 :0.97 ,test 교차 검증 정확도 :0.96 
학습레이블 분포 :  Counter({0: 34, 1: 33, 2: 33}) 
검사레이블 분포 :  Counter({1: 17, 2: 17, 0: 16})
train 평균 정확도 0.9700000000000001
test 평균 정확도 0.96


In [28]:
### Skf 모델 진행